# ``Box``

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import pySecDec as psd # Import pySecDec

In this notebook, we experiment with computing a 1-loop box integral:

$$
I_4^{D=4-2\epsilon}(0,0,0,0;s_{12},s_{23},m^2,m^2,m^2,m^2)
$$

### Generating the integral code

First, we will compute the first, ``U``, and second, ``F``, Symmanzik polynomials:

In [ ]:
li = psd.LoopIntegralFromGraph(
    internal_lines = [['m',[1,2]],['m',[2,3]],['m',[3,4]],['m',[4,1]]],
    external_lines = [['p1',1],['p2',2],['p3',3],['p4',4]],
    replacement_rules = [
        ('p1*p1', 0),
        ('p2*p2', 0),
        ('p3*p3', 0),
        ('p4*p4', 0),
        ('p1*p2', 's12/2'),
        ('p3*p4', 's12/2'),
        ('p2*p3', 's23/2'),
        ('p1*p4', 's23/2'),
        ('p2*p4', '-s12/2-s23/2'),
        ('p1*p3', '-s12/2-s23/2'),
        ('m**2', 'msq')
    ]
)
print('U:',li.exponentiated_U)
print('F:',li.exponentiated_F)

We now generate the c++ code for the box integral, using ``loop_package``:

In [ ]:
! rm -rf box
Mandelstam_symbols = ['s12','s23']
mass_symbols = ['msq']
psd.loop_package(
    name = 'box',
    loop_integral = li,
    real_parameters = Mandelstam_symbols + mass_symbols,
    requested_orders = [0],
    decomposition_method = 'geometric'
)

Next, we build the `disteval` version of the integral code, ready for numerical integration:

In [ ]:
! make disteval -C box > /dev/null 2>&1

### Evaluating the integral and manipulating the result

We can now load the pre-built integral code:

In [ ]:
box = psd.integral_interface.DistevalLibrary('box/disteval/box.json')

We begin by evaluating our integral at the point ``s12=4.1, s23=-2.2, msq=1``:

In [ ]:
result = box(parameters={'s12': 4.1, 's23': -2.2, 'msq': 1.}, format='json')
print(result)

We have requested that the library returns the result in `json` format, this provides us with a dictionary of the coefficients for each order of each regulator. 
We select the finite (`eps^0`) part of the box integral and print the result and its numerical uncertainty:

In [ ]:
finite_part = result['sums']['box'][(0,)]
print('res:', finite_part[0]) # Result
print('err:', finite_part[1]) # Numerical uncertainty 

### Plotting the result for different values of the real_parameters

We can also numerically compute our integral at several points, by repeatedly calling the integral code:

In [ ]:
xx = []
yy_re = []
yy_im = []
for ss in np.arange(0.01,10.01,1):
    result = box(parameters={'s12': ss, 's23': -2.2, 'msq': 1}, format='json')
    finite_part = result['sums']['box'][(0,)]
    print(ss, finite_part)
    xx.append(ss)
    yy_re.append(finite_part[0].real)
    yy_im.append(finite_part[0].imag)

In [ ]:
plt.plot(xx,yy_re,marker='o')
plt.plot(xx,yy_im,marker='x')